In [8]:
import os
with open("/home/billxbf/Documents/myks/openai.key", "r") as f:
    keys = f.readlines()
    os.environ["OPENAI_API_KEY"] = keys[0].strip()
with open("/home/billxbf/Documents/myks/serpapi.key", "r") as f:
    keys = f.readlines()
    os.environ["SERPAPI_API_KEY"] = keys[0].strip()
with open("/home/billxbf/Documents/myks/wolfram.key", "r") as f:
    keys = f.readlines()
    os.environ["WOLFRAM_ALPHA_APPID"] = keys[0].strip()

In [9]:
from utils.DataLoader import DataLoader
from utils.Evaluator import Evaluator
from algos.PWS import *
from algos.react import ReactBase, ReactExtraTool
from algos.notool import IO, CoT
from prompts import fewshots

In [10]:
def save_data(dataset, data, save_path):
    dataset["label"] = data["label"]
    dataset["preds"] = data["preds"]
    dataset["em"]  = data["em"]
    dataset["f1"] = data["f1"]
    dataset["acc"] = data["acc"]
    dataset["wall_time"] = data["wall_time"]
    dataset["total_tokens"] = data["total_tokens"]
    dataset["steps"] = data["steps"]
    dataset["tool_cost"] = data["tool_cost"]
    dataset["token_cost"] = data["token_cost"]
    dataset["total_cost"] = data["total_cost"]
    dataset.to_csv(save_path, index=False)
    return dataset

In [11]:
EVAL_LLM = "text-davinci-003"
EVAL_DATASET = "physics_question"
SEED = 2024
SAVE_DIR = "results/physics_question_{}_{}.csv"

## Standard IO

In [45]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(sample_size=None)
io = IO(model_name=EVAL_LLM)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=io)

In [46]:
response, data = eval.run()
df = save_data(dataset, data, SAVE_DIR.format("io", EVAL_LLM))
response


******************* Start Evaluation *******************



100%|██████████| 53/53 [02:15<00:00,  2.56s/it]


{'avg_em': 0.0,
 'avg_f1': 0.15125786163522012,
 'avg_acc': 0.5094339622641509,
 'avg_wall_time': 0.7276954425955718,
 'avg_total_tokens': 90.47169811320755,
 'avg_total_cost': 0.001809433962264151,
 'avg_steps': 1.0,
 'avg_token_cost': 0.001809433962264151,
 'avg_tool_cost': 0.0}

In [47]:
df.head()

,input,target,label,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,"During a recent morning run, Ken averaged a sp...","Can be one of ['7187 m', '7 km']","Can be one of ['7187 m', '7 km']",118.45 m,False,0.200000,1,0.582923,74,1,0,0.00148,0.00148
1,"Determine the momentum of a 360,000-kg passeng...",Can be one of ['540000 kg*m/s'],Can be one of ['540000 kg*m/s'],"540,000 kg m/s",False,0.222222,1,0.704528,48,1,0,0.00096,0.00096
2,The historic Stanley Center for the Arts in Ut...,Can be one of ['7974 N'],Can be one of ['7974 N'],Tension = 8062.9 N,False,0.222222,0,0.689390,100,1,0,0.00200,0.00200
3,A fire breaks out on a high rise building and ...,"Can be one of ['0.257 m/s/s', '0.26 m/s/s', '0...","Can be one of ['0.257 m/s/s', '0.26 m/s/s', '0...",2.6 m/s^2,False,0.000000,0,0.870070,161,1,0,0.00322,0.00322
4,Joss is parasailing. She sits in a seat harnes...,"Can be one of ['133 N', '133.0 N']","Can be one of ['133 N', '133.0 N']",162.88 N,False,0.200000,1,0.767975,72,1,0,0.00144,0.00144


## CoT

In [6]:
len('''Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types:
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.''') /4

129.75

In [5]:
import json
with open("data/prompts_naive.json") as f:
    prompts = json.load(f)
len(prompts["webthink_simple6"]) / 4

1473.25

In [2]:
from prompts import fewshots
len(fewshots.HOTPOTQA_REACT)

6338

In [50]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(None)
cot = CoT(fewshot=fewshots.TRIVIAQA_COT, model_name=EVAL_LLM)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=cot)

In [51]:
response, data = eval.run()
df = save_data(dataset, data, SAVE_DIR.format("cot", EVAL_LLM))
response


******************* Start Evaluation *******************



100%|██████████| 53/53 [06:00<00:00,  6.81s/it]


{'avg_em': 0.0,
 'avg_f1': 0.1687992978780992,
 'avg_acc': 0.5849056603773585,
 'avg_wall_time': 5.126167553775716,
 'avg_total_tokens': 273.45283018867923,
 'avg_total_cost': 0.005469056603773585,
 'avg_steps': 2.3773584905660377,
 'avg_token_cost': 0.005469056603773585,
 'avg_tool_cost': 0.0}

In [25]:
df.head()

,input,target,label,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,A massive light hangs over the table in Jeremy...,"Can be one of ['11.46 kg', '11.5 kg', '12 kg']","Can be one of ['11.46 kg', '11.5 kg', '12 kg']",The mass of the light is 11.0 kg.,False,0.250000,1,13.610541,398,3,0,0.000796,0.000796
1,Aaron Agin nodded off while driving home from ...,"Can be one of ['7.91 m/s', '7.9 m/s']","Can be one of ['7.91 m/s', '7.9 m/s']",vfinal = (1650 kg * 18.5 m/s,False,0.153846,0,14.639971,469,3,0,0.000938,0.000938
2,A California Condor is approaching its nest wi...,Can be one of ['31.3 m/s'],Can be one of ['31.3 m/s'],The speed of the carrion upon hitting the out...,False,0.266667,0,9.287838,350,2,0,0.000700,0.000700
3,"While playing basketball in PE class, Logan lo...",Can be one of ['3120 N'],Can be one of ['3120 N'],-40 m/s^2 (negative sign indicates deceleration),False,0.000000,0,7.399226,285,2,0,0.000570,0.000570
4,Renatta Gass is out with her friends. Misfortu...,Can be one of ['97680 J'],Can be one of ['97680 J'],"97,680 J",False,0.500000,1,4.352521,233,3,0,0.000466,0.000466


## REACT

In [52]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(None)
react = ReactExtraTool(model_name=EVAL_LLM, available_tools=["Google", "Wikipedia", "WolframAlpha", "Calculator", "LLM"], fewshot=fewshots.GSM8K_REACT, verbose=False)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=react)

In [53]:
response, data = eval.run()
df = save_data(dataset, data, SAVE_DIR.format("react", EVAL_LLM))
response


******************* Start Evaluation *******************



100%|██████████| 53/53 [08:49<00:00,  9.99s/it]


{'avg_em': 0.0,
 'avg_f1': 0.1578707593851586,
 'avg_acc': 0.5849056603773585,
 'avg_wall_time': 8.246861795209488,
 'avg_total_tokens': 1874.188679245283,
 'avg_total_cost': 0.03732449056603774,
 'avg_steps': 2.547169811320755,
 'avg_token_cost': 0.03732449056603774,
 'avg_tool_cost': 0.0}

In [9]:
df.head(10)

,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,In what city and state was the vocalist for E...,"Redwood City, California",nan,False,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
1,"What was the birth date of the director of ""Th...","December 28, 1888March",nan,False,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
2,What is the name of the cognac house that make...,Jas Hennessy & Co.,Hennessy,False,0.500000,1,8.891754,5092.0,3.0,0.0,0.010184,0.010184
3,The Wisconsin Badgers football team has had a ...,The Iron Horse,the Iron Horse or the Horse,False,0.666667,0,12.183595,7105.0,4.0,0.0,0.014210,0.014210
4,"Which film director is older, Jan Troell or Ag...",Agnès Varda,Jan Troell,False,0.000000,0,9.318587,5263.0,3.0,0.0,0.010526,0.010526
5,"Foshan Lingnan Mingzhu Gymnasium, where Boxing...",2006,2006,True,1.000000,1,7.464730,3284.0,2.0,0.0,0.006568,0.006568
6,In between Parenting and Cook's Illustrated wh...,Parenting,Parenting,True,1.000000,1,10.895709,7210.0,4.0,0.0,0.014420,0.014420
7,"In the year 2016, how many passengers used the...",11.04 million,Agent stopped due to iteration limit or time l...,False,0.000000,0,33.010987,21835.0,9.0,0.0,0.043670,0.043670
8,What was the nationality of Robert Kardashian'...,American,Agent stopped due to iteration limit or time l...,False,0.000000,0,21.680504,14876.0,9.0,0.0,0.029752,0.029752
9,In what Ohio city does the highway which the n...,Cincinnati,Agent stopped due to iteration limit or time l...,False,0.000000,0,30.278475,17950.0,9.0,0.0,0.035900,0.035900


## PWSBase

In [12]:
dataset = DataLoader(EVAL_DATASET, seed=SEED).load(None)
pws = PWS_Base(available_tools=["Google", "Wikipedia", "WolframAlpha", "Calculator", "LLM"], fewshot=fewshots.GSM8K_PWS, planner_model=EVAL_LLM, solver_model=EVAL_LLM)
eval = Evaluator(task=EVAL_DATASET, dataset=dataset, algo=pws)

In [13]:
print(pws.run(dataset.input[0])["solver_log"])

Solve the following task or problem. To assist you, we provide some plans and corresponding evidences that might be helpful. Notice that some of these information contain noise so you should trust them with caution.

During a recent morning run, Ken averaged a speed of 5.9 m/s for 14.9 minutes. Ken then averaged a speed of 6.25 m/s for 5.1 minutes. Determine the total distance which Ken ran during his 20 minute jog.
Plan: Calculate the total distance ran by multiplying the average speed by the total time.
Evidence:
119.78500000000001

Now begin to solve the task or problem. Respond with the answer directly with no extra words.

During a recent morning run, Ken averaged a speed of 5.9 m/s for 14.9 minutes. Ken then averaged a speed of 6.25 m/s for 5.1 minutes. Determine the total distance which Ken ran during his 20 minute jog.

119.78500000000001


In [43]:
response, data = eval.run()
df = save_data(dataset, data, SAVE_DIR.format("pws", EVAL_LLM))
response


******************* Start Evaluation *******************



100%|██████████| 53/53 [10:56<00:00, 12.38s/it]


{'avg_em': 0.0,
 'avg_f1': 0.14031239139452237,
 'avg_acc': 0.660377358490566,
 'avg_wall_time': 10.084138060515782,
 'avg_total_tokens': 1225.754716981132,
 'avg_total_cost': 0.00706698113207547,
 'avg_steps': 2.5660377358490565,
 'avg_token_cost': 0.00706698113207547,
 'avg_tool_cost': 0.0}

In [13]:
df.head(10)

,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,Noble is a 2014 film written and directed by S...,The Cripple of Inishmaan,Normal People (miniseries).,False,0.0,0,7.235505,1206,4,0.0,0.02412,0.02412
1,Who was the English clergyman and Archbishop o...,Henry Beaufort,Otto von Ziegenhain led the Battle of Tachov.,False,0.0,0,9.427552,1235,5,0.0,0.02470,0.02470
2,Which fictional character in the Harry Potter ...,Rubeus Hagrid,Rubeus Hagrid.,True,1.0,1,5.455595,1460,3,0.0,0.02920,0.02920
3,What Classification is the album by UK band Se...,Trip hop,Britpop.,False,0.0,0,6.557354,1226,4,0.0,0.02452,0.02452
4,What actor from The Imitation Game also starre...,Mark Strong,Benedict Cumberbatch,False,0.0,0,16.167417,2319,6,0.0,0.04638,0.04638


,question,answer,preds,em,f1,acc,wall_time,total_tokens,steps,tool_cost,token_cost,total_cost
0,Noble is a 2014 film written and directed by S...,The Cripple of Inishmaan,Normal People (miniseries).,False,0.000000,0,7.235505,1206,4,0.0,0.02412,0.02412
1,Who was the English clergyman and Archbishop o...,Henry Beaufort,Otto von Ziegenhain led the Battle of Tachov.,False,0.000000,0,9.427552,1235,5,0.0,0.02470,0.02470
2,Which fictional character in the Harry Potter ...,Rubeus Hagrid,Rubeus Hagrid.,True,1.000000,1,5.455595,1460,3,0.0,0.02920,0.02920
3,What Classification is the album by UK band Se...,Trip hop,Britpop.,False,0.000000,0,6.557354,1226,4,0.0,0.02452,0.02452
4,What actor from The Imitation Game also starre...,Mark Strong,Benedict Cumberbatch,False,0.000000,0,16.167417,2319,6,0.0,0.04638,0.04638
5,This 2006 Iowa Hawkeyes football coach played ...,Kirk Ferentz,"Yes, Kirk Ferentz played linebacker for the Un...",False,0.307692,1,9.570546,1998,5,0.0,0.03996,0.03996
6,Ikivo Animator integrates with a software appl...,Adobe Systems,Hypermedia.,False,0.000000,0,4.108220,723,3,0.0,0.01446,0.01446
7,What American actress born in 1912 did the vio...,Marta Eggerth,Mary Pickford.,False,0.000000,0,11.671225,2403,5,0.0,0.04806,0.04806
8,"Which plant has more species, Teucrium or Atal...",Teucrium,Teucrium has more species.,False,0.400000,1,6.937577,1497,4,0.0,0.02994,0.02994
9,When was the British-American journalist and e...,3 November 1949,Unknown.,False,0.000000,0,8.877709,1484,5,0.0,0.02968,0.02968
